# Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams

This notebook implements the methods described in [Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4040303), to be presented at the [Computational Legal Studies Conference](https://cclaw.smu.edu.sg/events/computational-legal-studies-2022), March 2022, hosted by the Center for Computational Law at Singapore Management University. Here is a link to the presentation's [slide deck](https://docs.google.com/presentation/d/15fxG3zoZSdUfmxFuVQZd2_MrDe1Z1rcsXwBDIvFIy3U/edit?usp=sharing), including image credits. 

Some of the data presented here differs slightly from that found in the current SSRN article. Neither the p value or effect size have changed when measured to one significant figure, however. The difference comes from the fact that since the posting of the current SSRN version, it was decided that the numbers here would be more representative of the method's performance if instead of using one run of the algo over a single order of the input data the process was conducted over 100,000 pseudo-random shuffles. The older version of this repo can be found [here](https://github.com/colarusso/free-response-scoring/tree/24fccbc22817ed72687b3b35e0dcc461f724dc0b).

Paper Summary

> This paper presents a novel method for unsupervised machine scoring of short answer and essay question responses, relying solely on a sufficiently large set of responses to a common prompt, absent the need for pre-labeled sample answers—given said prompt is of a particular character. That is, for questions where “good” answers look similar, “wrong” answers are likely to be “wrong” in different ways. Consequently, when a collection of text embeddings for responses to a common prompt are placed in an appropriate feature space, the centroid of their placements can stand in for a model answer, providing a lodestar against which to measure individual responses. This paper examines the efficacy of this method and discusses potential applications.
>
>Current methods for the automated scoring of short answer and essay questions are poorly suited to spontaneous and idiosyncratic assessments. That is, the time saved in grading must be balanced against the time required for the training of a model. This includes tasks such as the creation of pre-labeled sample answers. This limits the utility of machine grading for single classes working with novel assessments. The method described here eliminates the need for the preparation of pre-labeled sample answers. It is the author’s hope that such a method may be leveraged to reduce the time needed to grade free response questions, promoting the increased adoption of formative assessment esp. in contexts like law school instruction which traditionally have relied almost exclusively on summative assessments.
>
>Ranking by the algorithm is found to be statistically significant when compared to a pseudo-random shuffle. To determine how similar a list’s order was to that produced by a human grader, the lowest number of neighbor swaps needed to transform the ordering of these lists into that of the human ordering was calculated. For a dataset including more than one thousand student answers to a set of thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors, the p-value for a paired t-test of the two populations’ swaps, with the pseudo-random group acting as the untreated group and the machine-grader acting as the treatment, came to 0.000001, allowing us to reject the null hypothesis that the machine's ordering is equivalent to a random shuffle. Additionally, the Cohen’s d for the number of swaps between the pseudo-random ordering and machine ordering was found to be large (i.e., 0.997).

This notebook was used to obtain the results referenced above. Student exam answers and their associated grade data were acquired and this research conducted after approval was granted by Suffolk University’s Office of Research and Sponsored Programs (ORSP) which oversees all human subject research at Suffolk University.

This notebook implements the primary steps of the method described in the paper, namely:  

1. Produce an embedding for each answer that captures as much of the relevant information as possible.
2. Find the centroid for all of the embeddings in your population of answers and impute the location of a  “correct” model answer.
3. Measure the distance between each answer’s embedding and the “correct” answer (e.g., the answers’ centroid or medoid).
4. Convert the answers’ distances from the model answer into z-scores for the population of answers.
5. Translate these z-scores into some known grading scale.
6. Order the answers according to this scale.
7. Compare these orderings to random orderings by seeing how many times you have to change their rankings to obtain the same ordering as one would get if they were ordered by their human-assigned grades.

_Note: The methods described here are the subject of USPTO Patent Application Serial No. 17/246,563._

## Contents

_These, and other internal links (e.g., "back to contents"), may not work if you are viewing the preview of this notebook on GitHub.com. Links to other notebooks, external sites, and exam questions should, however, be okay._

This notebook has three sections:

- [Data: Student Exam Answers](#Data:-Student-Exam-Answers)
- [Code:Code: Libraries and Novel Functions](#Code:-Libraries-and-Novel-Functions)
- [Results](#Results)


## Data: Student Exam Answers

More than one thousand exam answers were obtained as PDF files containing answers to thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors. Each PDF corresponded to a single student exam. These PDFs were parsed to extract their answers, their contents converted into XML of the following format.

```
<EXAM>
    <STUDENT id='ID'>00000000</STUDENT>
    <QUESTION id='Q1'>
        <![CDATA[
            text of written answer to question one
        ]]>
    </QUESTION>
    <QUESTION id='Q2'>
        <![CDATA[
            text of written answer to question two
        ]]>
    </QUESTION>
    <QUESTION id='Q3'>
        <![CDATA[
            text of written answer to question three
        ]]>
    </QUESTION>
</EXAM>
```

These translated files were reviewed by hand and reformatted as needed to correct any formatting errors. Each XML file was then read into a csv file for its associated exam as a single row with their columns corresponding to each of its elements (e.g., ID, Q1, Q2). Additionally, a column stating the number of words contained in each question was appended to the csv file (e.g., size_Q1, size_Q2). E.g., 

|ID|Q1|Q2|size_Q1|size_Q2|
|--|--|--|-------|-------|
|00001|text of 1's ans to q1|text of 1's ans to q2|6|6|
|00002|text of 2's ans to q1|text of 2's ans to q2|6|6|

Instructors also provided scores for each exam question. These were placed in a csv for each exam with the scores on each question associated to the exam ID. E.g., 

|ID|Q1|Q2|
|--|--|--|
|00001|96|93|
|00002|83|89|

A PDF did not extist for every exam as some exams were hand-written. Also, for the _Property Instructor A_ exam, the IDs of two PDFs did not macth any of the IDs provided in the score sheet. To address such issues, you will see that we merge the two data tables above using the default behavior of Panda's merge method (i.e., we kept only the intersection of ID, those found in both files). Consequently, we did not score the exams for which no PDF existed or those for which we could not find a score on the intructor-provided list of scores.

To see the code used to assist in this processing see the following notebook: [Prep Exams](Prep%20Exams.ipynb).

In keeping with the wishes of those instructors who provided exams, only three of the exams are available here for review (i.e., [property_instructor_A](https://colarusso.github.io/free-response-scoring/exam_questions/property_instructor_A.docx), [property_instructor_B](https://colarusso.github.io/free-response-scoring/exam_questions/property_instructor_B.docx), and [crim_instructor_E](https://colarusso.github.io/free-response-scoring/exam_questions/crim_instructor_E.docx)). Another two are on file with the author and may be shared upon request and the assent of their authors. In keeping with the author's wishes, the remaining exam will not be shared. Subject to constraints imposed by the Family Educational Rights and Privacy Act (FERPA), at least three of the answer sets may be shared upon request. One of these is linked to an exam requiering instructor assent to be shared. I am in the process of reviewing and cleaning these data for PII. You can track my progress [here](https://twitter.com/Colarusso/status/1494781654237425666?s=20&t=g0Zt5f3wDnVys6oH3nVSGQ).

Please note that the exam questiones shared at the links above are docx files and have been redacted to exclude the instructor's name and text that does not include the scored question prompts (e.g., multiple choice questions). 

To facilitate retrieval of the exam answers which were stored in various folders, a list of dictionaries is defined. Each dictionary defines the folder name where csv files can be found as well as the names of its relevant columns for later consideration (i.e., their ID and those questions to be scored). To avoid sharing of this data publicly, as seen below, these files are not included in this repository and were located outside of this repository's folder (i.e., `../data/`). 

In [1]:
exams = [
            {"folder":"../data/property_instructor_A","columns":["ID","SHORT_ANS","Q1","Q2"]},
            {"folder":"../data/property_instructor_B","columns":["ID","Q1","Q2"]},
            {"folder":"../data/environ_instructor_B","columns":["ID","Q2"]},
            {"folder":"../data/PR_instructor_C","columns":["ID","Q1","Q2"]},
            {"folder":"../data/contracts_instructor_D","columns":["ID","Q1","Q2","Q3"]},
            {"folder":"../data/crim_instructor_E","columns":["ID","Q1","Q2"]}
          ]

[back to contents](#Contents)

## Code: Libraries and Novel Functions

### Python Libraries et al.

First we'll load the following libraries. These are needed to perform our scoring. 

In [2]:
import csv
import pandas as pd
import numpy as np
from numpy import var
from math import sqrt
import spacy
from scipy import stats
from sklearn.preprocessing import normalize

# https://spacy.io/models/en#en_core_web_lg
nlp1 = spacy.load('en_core_web_lg') 
# https://github.com/explosion/spacy-transformers/tree/88814f5f4be7f0d4c784d8500c558d9ba06b9a56
nlp2 = spacy.load("en_trf_distilbertbaseuncased_lg") 
nlp3 = spacy.load("en_trf_robertabase_lg") 

In [3]:
def score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1):

    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    M = []  
    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    R = []

    # for every exam in the list of exams  
    for exam in exams:

        print("\n=========================================================")
        print(exam["folder"])
        print("=========================================================\n")

        # load this exam's answer texts
        texts = pd.read_csv("%s/texts_redacted.csv"%exam["folder"], encoding="utf-8")
        # load this exam's grades
        actual = pd.read_csv("%s/actual_redacted.csv"%exam["folder"], encoding="utf-8")

        # print the relevant columns for this exam (i.e., ID and questions to score)
        print(exam["columns"])

        # score the questions listed above
        output_df = grade(texts[texts["ID"].isin(actual["ID"])][exam["columns"]],model,normv,score,goal)
        
        # below we want the average word counts for questions
        # so we need a df with only the questions actually used here
        # i.e., the inner merge. output_df doesn't have the texts columns
        # so we'll do a second merge here real quick to facilitate that count
        texts_df = actual.merge(texts, on="ID")

        # only include grades for those questions defined above
        actual = actual[[x for x in actual.columns if x in exam["columns"]]]

        # merge the machine-scored dataframe with that of the actual grades
        df_exam = actual.merge(output_df, on="ID")
        
        # display a preview of the two scores (first few rows of the merged dataframe)
        # I'm excluding the ID here to avoid sharing IDs in the cell's output
        display(df_exam[[x for x in list(df_exam.columns.values) if (x not in ["ID"])]].head())

        print("Number of entries:",len(df_exam))

        # for each of the questions that were scored
        for qn in [x for x in exam["columns"] if (x not in ["ID"])]:
            mean_words = int(round(texts_df["size_%s"%qn].mean()))
            print("\n%s\nMean Words:"%qn,mean_words)

            # create a dataframe with the grade and machine scores for this question
            df = df_exam[["ID", "%s_x"%qn, "%s_y"%qn]]

            # ideally the granularity of the machine and human scoring should be the same
            # to avoid false precision. It may be worth imposing this in some manner 
            # simmilar to this comment. 
            # df["%s_y"%qn] = round(df["%s_x"%qn].max()*(df["%s_y"%qn]/100))
            # another option would be to conpare rounded z scores from both samples 

            # create an list of these scores then count the swaps needed to bring them into 
            # agreement with the grades, averaged over `runs`. Append this to our list of 
            # swaps needed to move from machine-scored ordering to that of the graded exams. 
            # we're shuffling before sorting to address the fact that the original order of
            # rows can effect the swaps as the order of actual scores could be different
            # within the bins provided by the machine grade 
            tmp = []
            for x in range(runs):
                # shuffle df then sort answers by the machine scores
                # this is needed becaue the `actual` rows can effect the number 
                # of swaps, so we want to get a represnative sample over `runs`
                # note the use of `sample.` This is where we shuffle the base df
                df = df.sample(frac=1).sort_values(by='%s_y'%qn, ascending=True)
                
                arr_ = df[df.columns[1]].values.copy()
                n = len(arr_)
                M_o = countSwaps(arr_, n)
                tmp.append(M_o)

            M_o = np.array(tmp).mean()
            M.append(M_o)

            # we're going to do the same as the above but for pseudo-random shuffles.
            # we'll shuffle and count swaps for the number of times defined in `runs`
            # and average the results 
            tmp = []
            for x in range(runs):
                # note the use of `sample.` This is where we get our pseudo-random ordering
                arr_ = df[df.columns[1]].sample(frac=1).values.copy()
                n = len(arr_)
                R_o = countSwaps(arr_, n)
                tmp.append(R_o)

            R_o = np.array(tmp).mean()
            R.append(R_o)

            print("Algo score, swaps needed:",M_o)
            print("Swaps needed for first 10 out of %s pseudo-random runs:"%runs)
            print(tmp[:10])
            print("Pseudo-random, avergae swaps needed:",R_o)

    print("\n=========================================================")
    print("Swaps (Machine):")
    print(M)
    print("N:",len(M),"\tMean: ",np.mean(M),"\tVar: ",np.var(M))
    print("Avergae swaps (Pseudo-random):")
    print(R)
    print("N:",len(R),"\tMean: ",np.mean(R),"\tVar: ",np.var(R),"\n")

    # Print out a summary table of the percent difference between pseudo-random and machine swaps
    pdiff = []
    i = 0
    for x in R:
        if x != 0 and M[i] != 0:
            pdiff_ = 100*(x-M[i])/((x+M[i])/2)
            pdiff.append(pdiff_)
            print( i+1,")",x,">",M[i],"=>",pdiff_ )
        else:
            print("error")
        i = i + 1

    print("\nAvergae percent difference between swaps needed for pseudo-random and machine sorting:")
    print(np.array(pdiff).mean(),"\n")
        
    # run t test for the two populations of swaps
    sample_n = len(M)
    print(stats.ttest_rel(M[:sample_n], R[:sample_n]))
    # return the effect size
    print("Cohen's d: ",cohend(R[:sample_n], M[:sample_n]))

In [4]:
def grade(df,model,normv,score=0,goal="centroid"):
       
    df = df.copy()    
    
    # exclude the ID column to get only the answer texts
    questions = [x for x in list(df.columns.values) if (x not in ["ID"])]
    # vectorize the text of each answer
    for x in questions:
        df["%s_vec"%x] = df[x].apply(vectorize, args=(model,normv)) 
    
    # for each question
    for x in questions:
        
        # based on the `goal` parameter grade, against either the centroid or medoid
        # that is, set either as the model answer aginst which to grade others
        if goal=="centroid":
            # centroid 
            best_ans = df["%s_vec"%x].mean()
        else:
            # medoid 
            best_ans = df["%s_vec"%x].iloc[best_answer(df,x)]

        # measure each answer's distance from the model answer above
        df["%s_dist"%x] = df["%s_vec"%x].apply(distance, args=(best_ans,))
        
        # replace the text of the answer with its z-score
        df[x] =  df["%s_vec"%x].apply(z_score, args=(df,x,best_ans))
        
        # translate the z-score into a standard grade using
        # the translation defined by `score`
        if score==0:
            df[x] =  df[x].apply(score_0)
        elif score==1:
            df[x] =  df[x].apply(score_1)
            
    # create output df without the vectors or distance measures
    output_df = df[[ x for x in list(df.columns.values) if ("_vec" not in x) and ("_dist" not in x) ]].copy()
            
    return output_df

In [5]:
def vectorize(row,model='nlp1',normv=0): 

    # use a method for vectorization based on the `model` parameter
    # note: some methods can work across the entire text of a qustion
    # others requier vectorization at the sentence level
    if model=='nlp1':
        nlp = nlp1
        mode = 0
    elif model=='nlp2':
        nlp = nlp2
        mode = 1
    elif model=='nlp3':
        nlp = nlp3
        mode = 1
        
    try:
        if mode == 1:
            # vectorize at the sentence level
            doc = nlp(row)
            tmp_df = pd.DataFrame({'sent' : []})
            i=0 
            for sent in doc.sents:
                sent_vec = nlp(sent.string.strip()).vector
                # if normv=1 normalize each sentence vector
                if normv==1:
                    sent_vec = norm(sent_vec)
                if i==0:
                    tmp_df = pd.DataFrame([[sent_vec]],columns=["sent"])
                else:
                    tmp_df = tmp_df.append(pd.DataFrame([[sent_vec]],columns=["sent"]), ignore_index=True)
                i+=1
            # average sentence vectors 
            output = tmp_df["sent"].mean()
        else:
            # vectorize all input text together
            output = nlp(row).vector

        # if normv=1 normalize the answer's vector
        if normv==1:
            output = norm(output)  
    
        return output
    except:
        return np.NaN
    
def norm(row):
    # normalize vector
    matrix = row.reshape(1,-1).astype(np.float64)
    return normalize(matrix, axis=1, norm='l1')[0]

In [6]:
def best_answer(df,column):
    # Return the medoid (like the centroid, but confined to a member of the group) 
    # i.e., the best answer in the group of answers.
    tmp_df = df[[ "%s_vec"%column ]].copy()
    centroid = tmp_df["%s_vec"%column].mean()
    tmp_df["%s_dist"%column] = tmp_df["%s_vec"%column].apply(distance, args=(centroid,))
    tmp_df = tmp_df[tmp_df["%s_dist"%column]==tmp_df["%s_dist"%column].min()]
    return tmp_df.index[0]

def distance(row,centroid):
    # calculate the Euclidean distance between centroid and row
    # See https://stackoverflow.com/a/1401828
    return np.linalg.norm(centroid-row)

In [7]:
def z_score(row,df,x,best):
    # calculate the z-score for a given distance
    return -1*(np.linalg.norm(best-row)-df["%s_dist"%x].mean())/df["%s_dist"%x].std()

In [8]:
# Define grade bin boarders based on z-score
# --------------------------------------
# There's no standard mapping between z-scores and letter grades
# However, it's common for schools to set their own curve which 
# is in the same spirit. Therefor I have created a mapping where
# the average answer recieves a B. Others might choose differently

grade_bins = [
              2,    # A+
              1.5,  # A
              1,    # A-
              0.5,  # B+
              0,    # B
              -0.5, # B-
              -1,   # C+
              -1.5, # C
              -2,   # C-
              -2.5, # D+
              -3,   # D
              -3.5  # D-
             ]

# Assign numeric grades based on z-score
# -----------------------------------------
# This was the original z-score to percentage grade translation.
# I took the above and translated it into numberic grades
# with the scale bottoming out at 59 and topping out at 97
def score_0(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93+((row-grade_bins[1])/(grade_bins[0]-grade_bins[1]))*4
    elif row >= grade_bins[2]:
        grade = 90+((row-grade_bins[2])/(grade_bins[1]-grade_bins[2]))*3
    elif row >= grade_bins[3]:
        grade = 87+((row-grade_bins[3])/(grade_bins[2]-grade_bins[3]))*3
    elif row >= grade_bins[4]:
        grade = 83+((row-grade_bins[4])/(grade_bins[3]-grade_bins[4]))*4
    elif row >= grade_bins[5]:
        grade = 80+((row-grade_bins[5])/(grade_bins[4]-grade_bins[5]))*3
    elif row >= grade_bins[6]:
        grade = 77+((row-grade_bins[6])/(grade_bins[5]-grade_bins[6]))*3
    elif row >= grade_bins[7]:
        grade = 73+((row-grade_bins[7])/(grade_bins[6]-grade_bins[7]))*4
    elif row >= grade_bins[8]:
        grade = 70+((row-grade_bins[8])/(grade_bins[7]-grade_bins[8]))*3
    elif row >= grade_bins[9]:
        grade = 67+((row-grade_bins[9])/(grade_bins[8]-grade_bins[9]))*3
    elif row >= grade_bins[10]:
        grade = 63+((row-grade_bins[10])/(grade_bins[9]-grade_bins[10]))*4
    elif row >= grade_bins[11]:
        grade = 60+(row-grade_bins[11])*(3/(grade_bins[10]-grade_bins[11]))
    else:
        grade = 59
        
    # The final output is rounded to avoid false percision. A better 
    # approach is to bin all of the scores. This is done in score_1 below
    return round(grade)

# Assign numeric grades based on z-score
# -----------------------------------------
# Upon subsequent reflection, I realized that binned scores were preferable
# to a continuous score with dropoffs at 59 and 97. The above translation,
# however, was used on the first run of this code and so is included here
# for compleatness.  
def score_1(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93
    elif row >= grade_bins[2]:
        grade = 90
    elif row >= grade_bins[3]:
        grade = 87
    elif row >= grade_bins[4]:
        grade = 83
    elif row >= grade_bins[5]:
        grade = 80
    elif row >= grade_bins[6]:
        grade = 77
    elif row >= grade_bins[7]:
        grade = 73
    elif row >= grade_bins[8]:
        grade = 70
    elif row >= grade_bins[9]:
        grade = 67
    elif row >= grade_bins[10]:
        grade = 63
    elif row >= grade_bins[11]:
        grade = 60
    else:
        grade = 59
    return grade

In [9]:
# The following functions are used to count the number of adjacent swaps needed
# to change one ordering into another. It was written by Shivam Gupta
# See https://www.geeksforgeeks.org/number-swaps-sort-adjacent-swapping-allowed/
# -----------------------------------------------------
# python 3 program to count number of swaps required 
# to sort an array when only swapping of adjacent 
# elements is allowed. 
# include <bits/stdc++.h> 

#This function merges two sorted arrays and returns inversion count in the arrays.*/ 
def merge(arr, temp, left, mid, right): 
    inv_count = 0

    i = left #i is index for left subarray*/ 
    j = mid #i is index for right subarray*/ 
    k = left #i is index for resultant merged subarray*/ 
    while ((i <= mid - 1) and (j <= right)): 
        if (arr[i] <= arr[j]): 
            temp[k] = arr[i] 
            k += 1
            i += 1
        else: 
            temp[k] = arr[j] 
            k += 1
            j += 1

            #this is tricky -- see above explanation/ 
            # diagram for merge()*/ 
            inv_count = inv_count + (mid - i) 

    #Copy the remaining elements of left subarray 
    # (if there are any) to temp*/ 
    while (i <= mid - 1): 
        temp[k] = arr[i] 
        k += 1
        i += 1

    #Copy the remaining elements of right subarray 
    # (if there are any) to temp*/ 
    while (j <= right): 
        temp[k] = arr[j] 
        k += 1
        j += 1

    # Copy back the merged elements to original array*/ 
    for i in range(left,right+1,1): 
        arr[i] = temp[i] 

    return inv_count 

#An auxiliary recursive function that sorts the input 
# array and returns the number of inversions in the 
# array. */ 
def _mergeSort(arr, temp, left, right): 
    inv_count = 0
    if (right > left): 
        # Divide the array into two parts and call 
        #_mergeSortAndCountInv() 
        # for each of the parts */ 
        mid = int((right + left)/2) 

        #Inversion count will be sum of inversions in 
        # left-part, right-part and number of inversions 
        # in merging */ 
        inv_count = _mergeSort(arr, temp, left, mid) 
        inv_count += _mergeSort(arr, temp, mid+1, right) 

        # Merge the two parts*/ 
        inv_count += merge(arr, temp, left, mid+1, right) 

    return inv_count 

#This function sorts the input array and returns the 
#number of inversions in the array */ 
def countSwaps(arr, n): 
    temp = [0 for i in range(n)] 
    return _mergeSort(arr, temp, 0, n - 1) 

In [10]:
# The following function calculates Cohen d (the effect size)
# based on two populations. It was written by Jason Brownlee
# See https://machinelearningmastery.com/effect-size-measures-in-python/
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

[back to contents](#Contents)

# Results

The parameters below corresponds to those used when all of the exam data was first run through the above scoring method, with the exception of `runs`. To avoid biasing parameter selection, prior work on the method did NOT use the exam data used here. Note: the below cell has been re-run since this first run. Additionally, the current version makes use of the average number of swaps over 100,000 instances to provide a better representation of the method's performance.

In [11]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,83,75,84
1,9,30,22,87,90,86
2,6,27,9,69,72,83
3,1,1,4,82,72,79
4,13,26,25,91,85,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 989.49187
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1443, 1268, 1366, 1553, 1500, 1536, 1539, 1520, 1552, 1692]
Pseudo-random, avergae swaps needed: 1499.2635

Q1
Mean Words: 2048
Algo score, swaps needed: 1021.53089
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1571, 1465, 1592, 1454, 1451, 1656, 1704, 1527, 1640, 1678]
Pseudo-random, avergae swaps needed: 1566.17636

Q2
Mean Words: 947
Algo score, swaps needed: 830.0088
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1613, 1580, 1844, 1543, 1525, 1582, 1591, 1568, 1722, 1800]
Pseudo-random, avergae swaps needed: 1560.1296

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,88,88
1,18.0,20.0,85,80
2,16.5,18.0,59,89
3,18.0,22.5,82,90
4,24.0,22.5,84,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1289.48074
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1720, 1798, 2013, 1667, 1782, 1924, 1862, 1968, 1765, 1982]
Pseudo-random, avergae swaps needed: 1838.16739

Q2
Mean Words: 1546
Algo score, swaps needed: 1319.44584
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1590, 1748, 1852, 1944, 1830, 1819, 1734, 1795, 1595, 1746]
Pseudo-random, avergae swaps needed: 1757.34217

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 907.96902
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1107, 1426, 1236, 1337, 1401, 1363, 1499, 1309, 1352, 1298]
Pseudo-random, avergae swaps needed: 1324.92768

Q2
Mean Words: 570
Algo score, swaps needed: 813.94498
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1364, 1021, 1305, 1289, 1253, 1232, 1409, 1082, 1201, 1286]
Pseudo-random, avergae swaps needed: 1218.8786

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,82,89,85
1,44,29,30,85,84,89
2,38,36,20,81,84,81
3,47,16,21,79,81,81
4,38,25,31,89,81,89


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1175.99573
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1379, 1537, 1473, 1516, 1339, 1517, 1414, 1812, 1550, 1472]
Pseudo-random, avergae swaps needed: 1472.90819

Q2
Mean Words: 1050
Algo score, swaps needed: 995.51885
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1470, 1362, 1436, 1587, 1452, 1512, 1431, 1652, 1542, 1189]
Pseudo-random, avergae swaps needed: 1449.98639

Q3
Mean Words: 839
Algo score, swaps needed: 769.45228
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1164, 1435, 1196, 1344, 1430, 1322, 1494, 1392, 1590, 1682]
Pseudo-random, avergae swaps needed: 1458.82732

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,88
1,13.5,13.0,88,87
2,20.0,29.0,87,84
3,11.5,13.0,82,83
4,20.5,18.0,88,89


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1777.98812
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2150, 1974, 2039, 2124, 2171, 2141, 1981, 2166, 2509, 2027]
Pseudo-random, avergae swaps needed: 2045.47484

Q2
Mean Words: 2137
Algo score, swaps needed: 1635.97317
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2172, 1922, 1833, 2016, 1910, 2160, 1967, 1853, 2123, 1696]
Pseudo-random, avergae swaps needed: 1993.8371

Swaps (Machine):
[989.49187, 1021.53089, 830.0088, 1289.48074, 1319.44584, 120.99806, 907.96902, 813.94498, 1175.99573, 995.51885, 769.45228, 1777.98812, 1635.97317]
N: 13 	Mean:  1049.8306423076922 	Var:  160972.3340560062
Avergae swaps (Pseudo-random):
[1499.2635, 1566.17636, 1560.1296, 1838.16739, 1757.34217, 182.50397, 1324.92768, 1218.8786, 1472.90819, 1449.98639, 1458.82732, 2045.47484, 1993.8371]
N: 13 	Mean:  1489.8787007692308 	Var:  198389.97046703586 

1 ) 1499.2635 > 989.49187 => 40.9659893571621
2 ) 1566.1

The following cells present a variety of parameter selections, including a mix of vector normalization, grade translations, and "best answer" selections.

In [12]:
score_exams(exams,model='nlp1',normv=1,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,85,81,85
1,9,30,22,87,89,85
2,6,27,9,70,70,84
3,1,1,4,82,70,79
4,13,26,25,89,84,85


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 996.49944
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1563, 1437, 1557, 1636, 1575, 1558, 1597, 1563, 1509, 1453]
Pseudo-random, avergae swaps needed: 1499.05818

Q1
Mean Words: 2048
Algo score, swaps needed: 1055.01748
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1484, 1754, 1578, 1531, 1463, 1730, 1601, 1577, 1608, 1659]
Pseudo-random, avergae swaps needed: 1567.03827

Q2
Mean Words: 947
Algo score, swaps needed: 960.00272
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1588, 1671, 1378, 1579, 1618, 1683, 1501, 1517, 1470, 1308]
Pseudo-random, avergae swaps needed: 1559.54996

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,89
1,18.0,20.0,84,81
2,16.5,18.0,59,89
3,18.0,22.5,84,92
4,24.0,22.5,84,88


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1247.5142
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1816, 1746, 1987, 2169, 1870, 1989, 1775, 2138, 1960, 1901]
Pseudo-random, avergae swaps needed: 1837.61527

Q2
Mean Words: 1546
Algo score, swaps needed: 1214.00184
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1772, 1498, 1706, 1815, 1894, 1901, 1845, 1767, 1464, 1864]
Pseudo-random, avergae swaps needed: 1757.25254

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 909.02574
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1367, 1394, 1357, 1461, 1221, 1472, 1235, 1355, 1456, 1370]
Pseudo-random, avergae swaps needed: 1324.27231

Q2
Mean Words: 570
Algo score, swaps needed: 907.50728
Swaps needed for first 10 out of 100000 pseudo-random runs:
[998, 1200, 1209, 1282, 1200, 1202, 1439, 1340, 1298, 1213]
Pseudo-random, avergae swaps needed: 1219.76829

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,80,88,85
1,44,29,30,84,83,89
2,38,36,20,81,83,81
3,47,16,21,81,87,81
4,38,25,31,89,79,89


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1174.47396
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1494, 1421, 1452, 1464, 1385, 1334, 1401, 1157, 1406, 1691]
Pseudo-random, avergae swaps needed: 1472.65433

Q2
Mean Words: 1050
Algo score, swaps needed: 1041.00122
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1455, 1373, 1440, 1349, 1606, 1603, 1469, 1543, 1393, 1432]
Pseudo-random, avergae swaps needed: 1449.93829

Q3
Mean Words: 839
Algo score, swaps needed: 803.96645
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1375, 1488, 1590, 1543, 1290, 1455, 1453, 1304, 1459, 1501]
Pseudo-random, avergae swaps needed: 1458.05602

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,92,87
1,13.5,13.0,88,89
2,20.0,29.0,90,90
3,11.5,13.0,81,83
4,20.5,18.0,88,89


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1687.99523
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1963, 1917, 2243, 2089, 2153, 2264, 2023, 1967, 1980, 2310]
Pseudo-random, avergae swaps needed: 2045.0615

Q2
Mean Words: 2137
Algo score, swaps needed: 1579.01502
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2094, 2007, 1940, 1939, 1823, 1841, 2093, 2200, 2213, 2010]
Pseudo-random, avergae swaps needed: 1992.7924

Swaps (Machine):
[996.49944, 1055.01748, 960.00272, 1247.5142, 1214.00184, 114.00794, 909.02574, 907.50728, 1174.47396, 1041.00122, 803.96645, 1687.99523, 1579.01502]
N: 13 	Mean:  1053.079116923077 	Var:  135079.48589669733
Avergae swaps (Pseudo-random):
[1499.05818, 1567.03827, 1559.54996, 1837.61527, 1757.25254, 182.60975, 1324.27231, 1219.76829, 1472.65433, 1449.93829, 1458.05602, 2045.0615, 1992.7924]
N: 13 	Mean:  1489.6667007692306 	Var:  198207.1385893595 

1 ) 1499.05818 > 996.49944 => 40.27626819532221
2 ) 156

In [13]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,83,80,83
1,9,30,22,83,87,83
2,6,27,9,67,67,83
3,1,1,4,80,70,77
4,13,26,25,87,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1019.3981
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1426, 1451, 1497, 1442, 1452, 1494, 1562, 1293, 1548, 1389]
Pseudo-random, avergae swaps needed: 1498.85083

Q1
Mean Words: 2048
Algo score, swaps needed: 1070.53377
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1663, 1501, 1472, 1710, 1621, 1421, 1413, 1560, 1472, 1546]
Pseudo-random, avergae swaps needed: 1565.94968

Q2
Mean Words: 947
Algo score, swaps needed: 1034.87423
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1381, 1671, 1677, 1728, 1604, 1453, 1740, 1761, 1445, 1454]
Pseudo-random, avergae swaps needed: 1560.37929

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,87
1,18.0,20.0,83,80
2,16.5,18.0,59,87
3,18.0,22.5,83,90
4,24.0,22.5,83,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1274.96708
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1704, 1781, 1639, 1863, 1969, 1885, 1814, 1810, 1499, 1915]
Pseudo-random, avergae swaps needed: 1838.04514

Q2
Mean Words: 1546
Algo score, swaps needed: 1225.06252
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1847, 1560, 1842, 1894, 1601, 1940, 1631, 1777, 1729, 1939]
Pseudo-random, avergae swaps needed: 1757.58752

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 899.99841
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1426, 1206, 1218, 1171, 1512, 1019, 1228, 1344, 1489, 1366]
Pseudo-random, avergae swaps needed: 1324.53014

Q2
Mean Words: 570
Algo score, swaps needed: 1151.53591
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1196, 1232, 1231, 1158, 1187, 1239, 1122, 1206, 1139, 1092]
Pseudo-random, avergae swaps needed: 1219.55391

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,80,87,83
1,44,29,30,83,80,87
2,38,36,20,80,80,80
3,47,16,21,80,83,80
4,38,25,31,87,77,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1183.54727
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1299, 1541, 1348, 1639, 1511, 1477, 1525, 1559, 1508, 1382]
Pseudo-random, avergae swaps needed: 1472.55529

Q2
Mean Words: 1050
Algo score, swaps needed: 1061.37844
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1558, 1505, 1519, 1334, 1474, 1515, 1505, 1478, 1424, 1425]
Pseudo-random, avergae swaps needed: 1449.792

Q3
Mean Words: 839
Algo score, swaps needed: 800.40476
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1331, 1565, 1366, 1655, 1633, 1336, 1309, 1587, 1353, 1346]
Pseudo-random, avergae swaps needed: 1458.04389

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,87
1,13.5,13.0,87,87
2,20.0,29.0,87,87
3,11.5,13.0,80,83
4,20.5,18.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1704.57106
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2083, 2091, 1797, 2083, 1962, 1829, 2175, 2160, 2056, 2121]
Pseudo-random, avergae swaps needed: 2045.82565

Q2
Mean Words: 2137
Algo score, swaps needed: 1642.49874
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1913, 1909, 2112, 2070, 2007, 2170, 1861, 2002, 2035, 2223]
Pseudo-random, avergae swaps needed: 1995.49074

Swaps (Machine):
[1019.3981, 1070.53377, 1034.87423, 1274.96708, 1225.06252, 115.52697, 899.99841, 1151.53591, 1183.54727, 1061.37844, 800.40476, 1704.57106, 1642.49874]
N: 13 	Mean:  1091.0997892307691 	Var:  140516.49896111593
Avergae swaps (Pseudo-random):
[1498.85083, 1565.94968, 1560.37929, 1838.04514, 1757.58752, 182.53422, 1324.53014, 1219.55391, 1472.55529, 1449.792, 1458.04389, 2045.82565, 1995.49074]
N: 13 	Mean:  1489.9337153846152 	Var:  198533.2109508704 

1 ) 1498.85083 > 1019.3981 => 38.07826337486025
2

In [14]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,80,80,83
1,9,30,22,87,87,80
2,6,27,9,67,67,83
3,1,1,4,80,70,80
4,13,26,25,83,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1242.43613
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1412, 1435, 1386, 1417, 1563, 1454, 1617, 1504, 1597, 1459]
Pseudo-random, avergae swaps needed: 1499.55122

Q1
Mean Words: 2048
Algo score, swaps needed: 1165.45522
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1482, 1496, 1586, 1466, 1281, 1688, 1548, 1746, 1635, 1380]
Pseudo-random, avergae swaps needed: 1566.49256

Q2
Mean Words: 947
Algo score, swaps needed: 1350.96086
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1662, 1408, 1422, 1543, 1672, 1667, 1615, 1617, 1409, 1502]
Pseudo-random, avergae swaps needed: 1560.2925

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,90
1,18.0,20.0,80,80
2,16.5,18.0,59,83
3,18.0,22.5,83,87
4,24.0,22.5,83,83


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1257.8692
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1655, 1943, 1903, 1674, 1811, 1946, 1813, 1784, 1853, 1967]
Pseudo-random, avergae swaps needed: 1837.58315

Q2
Mean Words: 1546
Algo score, swaps needed: 1579.00927
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1842, 1812, 1985, 1438, 1553, 1783, 1827, 1680, 1689, 1683]
Pseudo-random, avergae swaps needed: 1756.40356

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 914.47371
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1398, 1353, 1443, 1212, 1185, 1369, 1260, 1282, 1451, 1253]
Pseudo-random, avergae swaps needed: 1324.40286

Q2
Mean Words: 570
Algo score, swaps needed: 1131.98515
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1239, 1291, 1145, 1142, 1256, 1147, 1321, 1063, 1155, 987]
Pseudo-random, avergae swaps needed: 1219.56355

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,80,83,83
1,44,29,30,80,77,87
2,38,36,20,77,83,80
3,47,16,21,83,87,80
4,38,25,31,87,73,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1299.48445
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1248, 1480, 1424, 1560, 1381, 1542, 1680, 1457, 1496, 1574]
Pseudo-random, avergae swaps needed: 1472.56011

Q2
Mean Words: 1050
Algo score, swaps needed: 1123.75333
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1421, 1521, 1596, 1625, 1545, 1702, 1407, 1509, 1466, 1500]
Pseudo-random, avergae swaps needed: 1449.38062

Q3
Mean Words: 839
Algo score, swaps needed: 923.92089
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1431, 1558, 1505, 1581, 1373, 1535, 1407, 1383, 1383, 1323]
Pseudo-random, avergae swaps needed: 1457.64031

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,87,83
1,13.5,13.0,83,87
2,20.0,29.0,87,87
3,11.5,13.0,80,83
4,20.5,18.0,87,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1882.35292
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2203, 2214, 2102, 2201, 2150, 2155, 2264, 2215, 1932, 2254]
Pseudo-random, avergae swaps needed: 2045.78578

Q2
Mean Words: 2137
Algo score, swaps needed: 1653.99452
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2026, 2055, 2190, 2084, 1819, 2273, 2060, 2135, 2131, 2216]
Pseudo-random, avergae swaps needed: 1994.65851

Swaps (Machine):
[1242.43613, 1165.45522, 1350.96086, 1257.8692, 1579.00927, 142.0105, 914.47371, 1131.98515, 1299.48445, 1123.75333, 923.92089, 1882.35292, 1653.99452]
N: 13 	Mean:  1205.2081653846153 	Var:  164735.96736347376
Avergae swaps (Pseudo-random):
[1499.55122, 1566.49256, 1560.2925, 1837.58315, 1756.40356, 182.43935, 1324.40286, 1219.56355, 1472.56011, 1449.38062, 1457.64031, 2045.78578, 1994.65851]
N: 13 	Mean:  1489.750313846154 	Var:  198424.6064283795 

1 ) 1499.55122 > 1242.43613 => 18.75392240595129
2

In [15]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,77,83,80
1,9,30,22,80,80,83
2,6,27,9,73,77,80
3,1,1,4,83,73,77
4,13,26,25,87,80,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1001.87025
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1504, 1626, 1577, 1666, 1454, 1328, 1687, 1625, 1529, 1669]
Pseudo-random, avergae swaps needed: 1499.60586

Q1
Mean Words: 2048
Algo score, swaps needed: 1192.99658
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1635, 1538, 1784, 1395, 1483, 1675, 1665, 1771, 1519, 1250]
Pseudo-random, avergae swaps needed: 1566.41842

Q2
Mean Words: 947
Algo score, swaps needed: 970.60954
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1677, 1655, 1536, 1670, 1546, 1710, 1553, 1760, 1806, 1608]
Pseudo-random, avergae swaps needed: 1559.46354

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,83
1,18.0,20.0,80,83
2,16.5,18.0,60,83
3,18.0,22.5,87,87
4,24.0,22.5,87,90


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1282.89934
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1865, 1848, 1637, 1723, 1928, 1720, 1735, 1646, 2006, 1840]
Pseudo-random, avergae swaps needed: 1837.4566

Q2
Mean Words: 1546
Algo score, swaps needed: 1230.00012
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1782, 1775, 1654, 1696, 1799, 1570, 1810, 1853, 1612, 1717]
Pseudo-random, avergae swaps needed: 1758.00574

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 919.05427
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1318, 1347, 1271, 1400, 1325, 1300, 1345, 1380, 1377, 1115]
Pseudo-random, avergae swaps needed: 1324.40838

Q2
Mean Words: 570
Algo score, swaps needed: 833.89411
Swaps needed for first 10 out of 100000 pseudo-random runs:
[989, 1071, 1252, 1399, 1266, 1189, 1249, 1106, 1231, 1332]
Pseudo-random, avergae swaps needed: 1219.83428

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,83,93,87
1,44,29,30,80,80,87
2,38,36,20,77,80,80
3,47,16,21,73,77,73
4,38,25,31,83,80,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1139.99621
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1412, 1758, 1359, 1602, 1457, 1541, 1325, 1509, 1590, 1511]
Pseudo-random, avergae swaps needed: 1472.95943

Q2
Mean Words: 1050
Algo score, swaps needed: 908.52419
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1449, 1411, 1570, 1431, 1303, 1464, 1641, 1193, 1437, 1747]
Pseudo-random, avergae swaps needed: 1449.79369

Q3
Mean Words: 839
Algo score, swaps needed: 857.4923
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1437, 1496, 1559, 1552, 1391, 1504, 1405, 1509, 1236, 1369]
Pseudo-random, avergae swaps needed: 1458.16598

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,87
1,13.5,13.0,80,87
2,20.0,29.0,90,83
3,11.5,13.0,77,83
4,20.5,18.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1746.09253
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2324, 2152, 2095, 2254, 2005, 2282, 2226, 1805, 2129, 2077]
Pseudo-random, avergae swaps needed: 2046.14633

Q2
Mean Words: 2137
Algo score, swaps needed: 1730.89581
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1902, 2087, 2001, 1961, 2058, 1993, 1789, 1992, 2130, 2006]
Pseudo-random, avergae swaps needed: 1993.9152

Swaps (Machine):
[1001.87025, 1192.99658, 970.60954, 1282.89934, 1230.00012, 91.0324, 919.05427, 833.89411, 1139.99621, 908.52419, 857.4923, 1746.09253, 1730.89581]
N: 13 	Mean:  1069.642896153846 	Var:  162115.29009007942
Avergae swaps (Pseudo-random):
[1499.60586, 1566.41842, 1559.46354, 1837.4566, 1758.00574, 182.37181, 1324.40838, 1219.83428, 1472.95943, 1449.79369, 1458.16598, 2046.14633, 1993.9152]
N: 13 	Mean:  1489.8880969230772 	Var:  198443.18616617093 

1 ) 1499.60586 > 1001.87025 => 39.79535187325853
2 ) 15

In [16]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,77,80,83
1,9,30,22,77,77,80
2,6,27,9,73,73,80
3,1,1,4,80,77,77
4,13,26,25,83,80,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1244.45804
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1425, 1303, 1572, 1428, 1634, 1320, 1494, 1658, 1522, 1509]
Pseudo-random, avergae swaps needed: 1499.85682

Q1
Mean Words: 2048
Algo score, swaps needed: 1322.01164
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1479, 1580, 1486, 1437, 1661, 1581, 1659, 1624, 1600, 1640]
Pseudo-random, avergae swaps needed: 1566.24958

Q2
Mean Words: 947
Algo score, swaps needed: 1267.87464
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1357, 1725, 1457, 1427, 1432, 1735, 1627, 1407, 1536, 1585]
Pseudo-random, avergae swaps needed: 1560.00336

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,87
1,18.0,20.0,77,83
2,16.5,18.0,67,83
3,18.0,22.5,87,87
4,24.0,22.5,87,83


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1277.55779
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1792, 1837, 1872, 1797, 1701, 1767, 1879, 1969, 1703, 1728]
Pseudo-random, avergae swaps needed: 1837.50391

Q2
Mean Words: 1546
Algo score, swaps needed: 1150.59557
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1664, 1687, 1734, 1623, 1840, 1906, 1861, 1673, 1700, 1871]
Pseudo-random, avergae swaps needed: 1757.63362

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 1007.47271
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1212, 1438, 1343, 1481, 1379, 1291, 1449, 1517, 1413, 1115]
Pseudo-random, avergae swaps needed: 1324.90532

Q2
Mean Words: 570
Algo score, swaps needed: 984.89115
Swaps needed for first 10 out of 100000 pseudo-random runs:
[947, 1374, 1217, 1362, 1123, 1154, 1272, 1107, 1413, 1356]
Pseudo-random, avergae swaps needed: 1219.73145

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,83,97,87
1,44,29,30,83,77,87
2,38,36,20,80,77,77
3,47,16,21,73,77,77
4,38,25,31,83,83,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1124.53718
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1715, 1639, 1506, 1464, 1513, 1434, 1621, 1589, 1500, 1563]
Pseudo-random, avergae swaps needed: 1472.27284

Q2
Mean Words: 1050
Algo score, swaps needed: 969.00679
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1249, 1470, 1726, 1604, 1461, 1548, 1612, 1582, 1343, 1305]
Pseudo-random, avergae swaps needed: 1449.93849

Q3
Mean Words: 839
Algo score, swaps needed: 774.97094
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1707, 1355, 1419, 1436, 1498, 1551, 1301, 1636, 1379, 1269]
Pseudo-random, avergae swaps needed: 1458.03545

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,87,83
1,13.5,13.0,83,83
2,20.0,29.0,87,83
3,11.5,13.0,73,83
4,20.5,18.0,87,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1790.54931
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2004, 1854, 1981, 2295, 2189, 2046, 2089, 1861, 1870, 1964]
Pseudo-random, avergae swaps needed: 2045.37647

Q2
Mean Words: 2137
Algo score, swaps needed: 1715.56764
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2171, 2000, 2234, 2221, 2048, 1947, 2142, 2013, 2121, 2110]
Pseudo-random, avergae swaps needed: 1993.73344

Swaps (Machine):
[1244.45804, 1322.01164, 1267.87464, 1277.55779, 1150.59557, 112.50113, 1007.47271, 984.89115, 1124.53718, 969.00679, 774.97094, 1790.54931, 1715.56764]
N: 13 	Mean:  1133.999579230769 	Var:  161042.35837586137
Avergae swaps (Pseudo-random):
[1499.85682, 1566.24958, 1560.00336, 1837.50391, 1757.63362, 182.3568, 1324.90532, 1219.73145, 1472.27284, 1449.93849, 1458.03545, 2045.37647, 1993.73344]
N: 13 	Mean:  1489.815196153846 	Var:  198350.86892576603 

1 ) 1499.85682 > 1244.45804 => 18.612935689165052

In [17]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,87,67,80
1,9,30,22,80,83,83
2,6,27,9,73,73,83
3,1,1,4,87,80,83
4,13,26,25,87,77,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1327.56784
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1429, 1502, 1726, 1596, 1427, 1520, 1570, 1585, 1393, 1506]
Pseudo-random, avergae swaps needed: 1499.01115

Q1
Mean Words: 2048
Algo score, swaps needed: 1283.87266
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1683, 1718, 1602, 1704, 1766, 1474, 1556, 1515, 1354, 1597]
Pseudo-random, avergae swaps needed: 1567.02842

Q2
Mean Words: 947
Algo score, swaps needed: 1217.32169
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1536, 1622, 1638, 1599, 1814, 1720, 1543, 1347, 1578, 1614]
Pseudo-random, avergae swaps needed: 1560.29184

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,87,90
1,18.0,20.0,60,67
2,16.5,18.0,63,83
3,18.0,22.5,77,87
4,24.0,22.5,87,90


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1578.46273
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1614, 1812, 1638, 2031, 1413, 1779, 2005, 1693, 1845, 1961]
Pseudo-random, avergae swaps needed: 1837.6727

Q2
Mean Words: 1546
Algo score, swaps needed: 1572.56844
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1667, 1620, 1607, 1850, 1705, 1623, 1676, 1452, 1946, 2019]
Pseudo-random, avergae swaps needed: 1757.48449

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 926.47932
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1248, 1378, 1198, 1159, 1401, 1291, 1406, 1280, 1452, 1456]
Pseudo-random, avergae swaps needed: 1324.80361

Q2
Mean Words: 570
Algo score, swaps needed: 1011.08692
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1043, 1400, 1227, 1048, 1278, 1368, 1469, 1237, 1259, 1389]
Pseudo-random, avergae swaps needed: 1219.78834

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,83,87,90
1,44,29,30,77,73,83
2,38,36,20,87,83,83
3,47,16,21,60,67,59
4,38,25,31,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1355.56455
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1469, 1533, 1424, 1269, 1432, 1602, 1455, 1474, 1636, 1543]
Pseudo-random, avergae swaps needed: 1472.84902

Q2
Mean Words: 1050
Algo score, swaps needed: 1163.52481
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1447, 1544, 1422, 1577, 1572, 1254, 1413, 1396, 1569, 1490]
Pseudo-random, avergae swaps needed: 1450.42346

Q3
Mean Words: 839
Algo score, swaps needed: 1059.45973
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1546, 1668, 1508, 1342, 1710, 1472, 1604, 1294, 1376, 1445]
Pseudo-random, avergae swaps needed: 1458.7563

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,87
1,13.5,13.0,83,87
2,20.0,29.0,80,77
3,11.5,13.0,87,80
4,20.5,18.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1719.09485
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2158, 2208, 1975, 2117, 1980, 2140, 2448, 2259, 2007, 1908]
Pseudo-random, avergae swaps needed: 2045.6147

Q2
Mean Words: 2137
Algo score, swaps needed: 1801.98901
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1874, 2017, 1943, 2068, 1985, 1718, 1934, 2213, 1966, 2053]
Pseudo-random, avergae swaps needed: 1994.20898

Swaps (Machine):
[1327.56784, 1283.87266, 1217.32169, 1578.46273, 1572.56844, 108.48545, 926.47932, 1011.08692, 1355.56455, 1163.52481, 1059.45973, 1719.09485, 1801.98901]
N: 13 	Mean:  1240.4213846153843 	Var:  174111.3265135405
Avergae swaps (Pseudo-random):
[1499.01115, 1567.02842, 1560.29184, 1837.6727, 1757.48449, 182.46128, 1324.80361, 1219.78834, 1472.84902, 1450.42346, 1458.7563, 2045.6147, 1994.20898]
N: 13 	Mean:  1490.03033 	Var:  198393.26756920136 

1 ) 1499.01115 > 1327.56784 => 12.130799146709863
2 ) 156

In [18]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,7,24,15,87,70,87
1,9,30,22,77,87,80
2,6,27,9,97,70,87
3,1,1,4,83,80,83
4,13,26,25,87,73,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1582.96844
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1625, 1679, 1487, 1616, 1401, 1751, 1451, 1428, 1335, 1383]
Pseudo-random, avergae swaps needed: 1498.88271

Q1
Mean Words: 2048
Algo score, swaps needed: 1485.04223
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1501, 1424, 1439, 1833, 1771, 1562, 1608, 1674, 1717, 1744]
Pseudo-random, avergae swaps needed: 1566.08606

Q2
Mean Words: 947
Algo score, swaps needed: 1520.59747
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1501, 1471, 1817, 1391, 1491, 1502, 1465, 1604, 1638, 1561]
Pseudo-random, avergae swaps needed: 1560.18474

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,15.0,21.0,83,87
1,18.0,20.0,63,67
2,16.5,18.0,70,83
3,18.0,22.5,77,90
4,24.0,22.5,90,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1566.56295
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1863, 1752, 2081, 1698, 1940, 1677, 1911, 1821, 1818, 1780]
Pseudo-random, avergae swaps needed: 1838.04316

Q2
Mean Words: 1546
Algo score, swaps needed: 1489.93971
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1902, 1810, 1646, 1585, 1772, 1648, 1581, 1650, 1641, 1753]
Pseudo-random, avergae swaps needed: 1757.35633

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 875.54217
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1281, 1174, 1365, 1431, 1251, 1150, 1165, 1263, 1223, 1355]
Pseudo-random, avergae swaps needed: 1324.55733

Q2
Mean Words: 570
Algo score, swaps needed: 945.98401
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1161, 1390, 1203, 1047, 1044, 1385, 1334, 1314, 1287, 1308]
Pseudo-random, avergae swaps needed: 1219.72133

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,55,40,41,80,87,97
1,44,29,30,77,73,80
2,38,36,20,87,83,83
3,47,16,21,60,67,59
4,38,25,31,83,83,90


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1298.50714
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1492, 1291, 1516, 1477, 1369, 1428, 1487, 1663, 1512, 1493]
Pseudo-random, avergae swaps needed: 1472.57778

Q2
Mean Words: 1050
Algo score, swaps needed: 1141.52596
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1474, 1518, 1471, 1572, 1359, 1623, 1399, 1244, 1433, 1522]
Pseudo-random, avergae swaps needed: 1449.69754

Q3
Mean Words: 839
Algo score, swaps needed: 1101.58632
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1379, 1620, 1428, 1351, 1318, 1452, 1608, 1472, 1467, 1501]
Pseudo-random, avergae swaps needed: 1458.23417

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,28.5,10.0,90,87
1,13.5,13.0,83,87
2,20.0,29.0,77,80
3,11.5,13.0,83,80
4,20.5,18.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1798.09742
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2047, 1842, 1973, 2095, 1973, 1666, 1978, 2066, 2046, 2047]
Pseudo-random, avergae swaps needed: 2045.25146

Q2
Mean Words: 2137
Algo score, swaps needed: 1769.02132
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2075, 2026, 1900, 1920, 1977, 1750, 1962, 1925, 1981, 1934]
Pseudo-random, avergae swaps needed: 1994.3004

Swaps (Machine):
[1582.96844, 1485.04223, 1520.59747, 1566.56295, 1489.93971, 96.52846, 875.54217, 945.98401, 1298.50714, 1141.52596, 1101.58632, 1798.09742, 1769.02132]
N: 13 	Mean:  1282.4541230769232 	Var:  196346.73477781538
Avergae swaps (Pseudo-random):
[1498.88271, 1566.08606, 1560.18474, 1838.04316, 1757.35633, 182.5351, 1324.55733, 1219.72133, 1472.57778, 1449.69754, 1458.23417, 2045.25146, 1994.3004]
N: 13 	Mean:  1489.8021623076925 	Var:  198373.31514573662 

1 ) 1498.88271 > 1582.96844 => -5.456832657216426

[back to contents](#Contents)